In [1]:
# import os
# import datetime
# from tqdm.notebook import tqdm
# from obspy import read
# import h5py
# import csv
# import json
# import numpy as np
# import pandas as pd
# import multiprocessing
# from multiprocessing.pool import ThreadPool

# os.getpid()

# def process(station_name, overlap=0.3):

#     # ex) /home/volume/workspace/Project/Earthquake/data/mseed_data/PH05
#     DIR = os.getcwd()
#     data_path = os.path.join(DIR, 'data/mseed_data')
#     station_path = os.path.join(data_path, station_name)    
#     mseed_list = os.listdir(station_path)
#     mseed_dict = dict()

#     for mseed in tqdm(mseed_list):
#         # ex) /home/volume/workspace/Project/Earthquake/data/mseed_data/PH05/PH.PH05.00.BHE__20180226T000000Z__20180227T000000Z.mseed
#         file_path = os.path.join(station_path, mseed)
#         # ex) PH.PH05.00.BHE__20180226T000000Z__20180227T000000Z.mseed
#         file_name = mseed
#         # ex) PH.PH05.00.20180226T000000Z__20180227T000000Z.mseed
#         file_name_st = mseed[:11] + mseed[16:32]
#         # ex) E or N or Z
#         axis = file_path.split('.')[3].split('__')[0][2]

#         st = read(file_path)

#         if file_name_st not in mseed_dict.keys():
#             mseed_dict[file_name_st] = dict()
 
#         mseed_dict[file_name_st][axis] = {
#             'file_name' : file_name,
#             'mseed' : st
#         }
    
#     data = mseed_dict    
    
#     json_station = json.load(open(os.path.join(os.getcwd(),"json/PH_station.json")))
#     output_name = list(data.keys())[0].split('.')[1]
    
#     try:
#         os.remove(os.path.join(DIR, 'preprocessed_data/' + output_name + '.hdf5'))
#         os.remove(os.path.join(DIR, 'preprocessed_data/' + output_name + '.csv'))
#     except Exception:
#         pass
    
#     HDF = h5py.File(os.path.join(DIR, 'preprocessed_data/' + output_name + '.hdf5'), 'a')
#     HDF.create_group("data")
    
#     slide_estimates = list()
#     csvfile = open(os.path.join(DIR, 'preprocessed_data/' + output_name + '.csv'), 'w')
#     output_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#     output_writer.writerow(['trace_name', 'start_time'])
#     csvfile.flush()     

#     for mseed in tqdm(data.keys()):
#         st = data[mseed]['E']['mseed'].copy()
#         st2 = data[mseed]['N']['mseed'].copy()
#         st3 = data[mseed]['Z']['mseed'].copy()
        
#         st.detrend('demean')
#         st2.detrend('demean')
#         st3.detrend('demean')
        
        
#         st.append(st2[0])
#         st.append(st3[0])
        
#         st.filter('bandpass', freqmin = 1.0, freqmax = 45, corners=2, zerophase=True)
        
#         st.taper(max_percentage=0.001, type='cosine', max_length=2)
        
#         org_samplingRate = st[0].stats.sampling_rate
#         longest = st[0].stats.npts
#         start_time = st[0].stats.starttime
#         end_time = st[0].stats.endtime
#         tim_shift = int(60-(overlap*60))
            
#         for tt in st:
#             if tt.stats.npts > longest:
#                 longest = tt.stats.npts
#                 start_time = tt.stats.starttime
#                 end_time = tt.stats.endtime
#         st.trim(start_time, end_time, pad = True, fill_value=0)
#         slide_estimates.append((end_time - start_time)//tim_shift)
        
#         chanL = [st[0].stats.channel[-1], st[1].stats.channel[-1], st[2].stats.channel[-1]]
#         next_slice = start_time + 60
        
#         while next_slice <= end_time:
#             w=st.slice(start_time, next_slice)
#             npz_data = np.zeros([12000, 3])
            
#             npz_data[:, 0] = w[chanL.index('E')].data[:12000]
#             npz_data[:, 1] = w[chanL.index('N')].data[:12000]
#             npz_data[:, 2] = w[chanL.index('Z')].data[:12000]
            
#             tr_name = st[0].stats.station + '_' + st[0].stats.network + '_' + \
#                       st[0].stats.channel[:2] + '_' + str(start_time)
#             start_time_str = str(start_time).replace('T', ' ').replace('Z', '')
                
#             HDF = h5py.File(os.path.join(DIR, 'preprocessed_data/'+ output_name +'.hdf5'), 'r')
#             dsF = HDF.create_dataset('data/'+tr_name, npz_data.shape, data = npz_data, dtype= np.float32)
#             dsF.attrs["trace_name"] = tr_name
#             dsF.attrs["receiver_code"] = output_name
#             dsF.attrs["network_code"] = json_station[output_name]['network']
#             dsF.attrs["receiver_latitude"] = json_station[output_name]['coords'][0]
#             dsF.attrs["receiver_longitude"] = json_station[output_name]['coords'][1]
#             dsF.attrs["receiver_elevation_m"] = json_station[output_name]['coords'][2]
#             dsF.attrs['trace_start_time'] = start_time_str
            
#             output_writer.writerow([str(tr_name), start_time_str])  
            
#             HDF.flush()
#             csvfile.flush()
            
#             start_time = start_time + tim_shift
#             next_slice = next_slice + tim_shift 
            
# #             break
# #         break
    
#         st1, st2, st3 = None, None, None
                
#     HDF.close() 
#     dd = pd.read_csv(os.path.join(DIR, 'preprocessed_data/' + output_name+".csv"))
    
#     return 1

# def preprocess(station_num, cpu=16):

#     DIR = os.getcwd()
#     data_path = os.path.join(DIR, 'data/mseed_data')
#     station_list = sorted(os.listdir(data_path))[:station_num]
    
#     n_processor = multiprocessing.cpu_count()
    
#     print('available cpu count : ', n_processor)
#     print('select cpu count : ', cpu)
    
#     with ThreadPool(cpu) as p:
#         p.map(process, station_list)
    
#     return 1
# preprocess(1, 1)

In [2]:
import os
import datetime
from tqdm.notebook import tqdm
from obspy import read

import h5py
import csv
import json
import numpy as np
import pandas as pd
import pickle

import multiprocessing
from multiprocessing.pool import ThreadPool
import gc

print(os.getpid())

DIR = os.getcwd()
data_path = os.path.join(DIR, 'data/mseed_data')

station_list = sorted(os.listdir(data_path))

data_tree = dict()

for station_name in tqdm(station_list):
    station_path = os.path.join(data_path, station_name)
    mseed_list = os.listdir(station_path)
    data_tree[station_name] = dict()
    
    for mseed in mseed_list:
        mseed_group = mseed[:11] + mseed[16:32]
        if mseed_group not in data_tree[station_name].keys():
            data_tree[station_name][mseed_group] = list()
        
        data_tree[station_name][mseed_group].append(mseed)
        
for station_name in data_tree.keys():
    for mseed_group in sorted(data_tree[station_name].keys()):
        if len(data_tree[station_name][mseed_group]) != 3:
            data_tree[station_name].pop(mseed_group)
            
del station_name, station_path, mseed, mseed_group

19848


In [3]:
import tracemalloc

tracemalloc.start()

# ... run your application ...

def preprocessing(file_list, window_size=6000, overlap=0.3):
    preprocessed_data = list()
    for file in file_list:
        file_path = os.path.join(data_path, file[3:7], file)
        if file.split('__')[0][-1] == 'E':
            st1 = read(file_path)
        if file.split('__')[0][-1] == 'N':
            st2 = read(file_path)
        if file.split('__')[0][-1] == 'Z':
            st3 = read(file_path)
            
        
    st1.detrend('demean')
    st2.detrend('demean')
    st3.detrend('demean')
    st1.append(st2[0])
    st1.append(st3[0])

    st1.filter('bandpass', freqmin = 1.0, freqmax = 45, corners=2, zerophase=True)
    st1.resample(100)
    st1.taper(max_percentage=0.001, type='cosine', max_length=2)

    org_samplingRate = st1[0].stats.sampling_rate
    longest = st1[0].stats.npts
    start_time = st1[0].stats.starttime
    end_time = st1[0].stats.endtime

    for tt in st1:
        if tt.stats.npts > longest:
            longest = tt.stats.npts
            start_time = tt.stats.starttime
            end_time = tt.stats.endtime
    st1.trim(start_time, end_time, pad = True, fill_value=0)

    next_slice = start_time + 60
    tim_shift = int(60-(overlap*60))

    while next_slice <= end_time:
        w=st1.slice(start_time, next_slice)
        npz_data = np.zeros([window_size, 3])

        npz_data[:, 0] = w[0].data[:window_size]
        npz_data[:, 1] = w[1].data[:window_size]
        npz_data[:, 2] = w[2].data[:window_size]

        
        tr_name = st1[0].stats.station + '_' + st1[0].stats.network + '_' + \
                  st1[0].stats.channel[:2] + '_' + str(start_time)
        start_time_str = str(start_time).replace('T', ' ').replace('Z', '')

        preprocessed_data.append({
            'trace_start_time': datetime.datetime.strptime(str(start_time), "%Y-%m-%dT%H:%M:%S.%fZ"),
            'trace_end_time': datetime.datetime.strptime(str(next_slice), "%Y-%m-%dT%H:%M:%S.%fZ"),
            'data': npz_data,
            'p_arv': None,
            's_arv': None,
            'p_label': 0,
            's_label': 0
        })
        
        start_time = start_time + tim_shift
        next_slice = next_slice + tim_shift
    
    return preprocessed_data
    
for station_name in list(data_tree.keys())[:13]:
    print(station_name)
    dir_path = os.path.join(DIR, 'data/preprocessed_data', station_name)
    
    if not os.path.isdir(dir_path):
        os.mkdir(dir_path)
        
    mseed_groups = sorted(list(data_tree[station_name].keys()))
    
    
    for idx, mseed_group in tqdm(enumerate(mseed_groups)):
        if mseed_group[11:19] != '20171115':
            continue
            
        print('found ', mseed_group[11:19])
        preprocessed_data = list()
            
        data = preprocessing(data_tree[station_name][mseed_group]).copy()
        preprocessed_data = preprocessed_data.copy() + data.copy()
        
        with open(os.path.join(dir_path, '{}.pkl'.format(mseed_group)), 'wb') as f:
            pickle.dump(preprocessed_data, f)
            
        if idx == 10:
            break

# ... finish application ...
snapshot = tracemalloc.take_snapshot()
top_stats = snapshot.statistics('lineno')

print("[ Top 10 ]")
for stat in top_stats[:10]:
    print(stat)

PH01


found  20171115
found  20171115

PH02


found  20171115

PH03


found  20171115
found  20171115

PH04


found  20171115
found  20171115

PH05


...

found  20171115
found  20171115

PH10


found  20171115

PH11



PH12



PH13



[ Top 10 ]
<ipython-input-3-098e1d5bbe1a>:46: size=226 MiB, count=4938, average=46.9 KiB
<__array_function__ internals>:6: size=58.5 MiB, count=29, average=2065 KiB
/usr/local/lib/python3.6/dist-packages/scipy/fft/_pocketfft/basic.py:251: size=29.2 MiB, count=2, average=14.6 MiB
<frozen importlib._bootstrap_external>:487: size=3050 KiB, count=28003, average=112 B
<ipython-input-3-098e1d5bbe1a>:64: size=484 KiB, count=1647, average=301 B
<ipython-input-3-098e1d5bbe1a>:93: size=394 KiB, count=5857, average=69 B
/usr/lib/python3.6/email/feedparser.py:469: size=214 KiB, count=24, average=9112 B
/usr/local/lib/python3.6/dist-packages/obspy/core/utcdatetime.py:1056: size=187 KiB, count=1994, average=96 B
/usr/local/lib/python3.6/dist-packages/obspy/core/util/attribdict.py:67: size=152 KiB, count=2159, average=72 B
/usr/lib/python3.6/_strptime.py:576: size=129 KiB, count=3292, average=40 B
